In [20]:
import os
os.getcwd()

'/content'

In [21]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.models import mobilenet_v2
from torchvision.datasets import FashionMNIST, SVHN
from PIL import Image
import numpy as np
import cv2
import os
from tqdm import tqdm

class CAM:
    def __init__(self, model, device, preprocess, layer_name=None):
        self.layer_name = layer_name if layer_name is not None else self._get_layer_name(model)
        self.model = model.eval().to(device)
        self.device = device
        self.prep = preprocess
        self.feature = {}
        self._register_hook()

    def _get_layer_name(self, model):
        layer_name = None
        last_name = None
        for name, module in model.named_modules():
            if hasattr(module, 'inplace'):
                module.inplace = False
            if isinstance(module, (nn.AdaptiveAvgPool2d, nn.AvgPool2d)):
                layer_name = last_name
            last_name = name

        if layer_name is None:
            raise ValueError('No appropriate layer found. Specify a layer for heatmap.')
        return layer_name

    def _forward_hook(self, module, x, y):
        self.feature['output'] = y

    def _register_hook(self):
        for name, module in self.model.named_modules():
            if name == self.layer_name:
                module.register_forward_hook(self._forward_hook)
                break
        else:
            raise ValueError(f'No layer named "{self.layer_name}" in the model')

    def _check(self, feature):
        if feature.ndim != 4 or feature.shape[2] * feature.shape[3] == 1:
            raise ValueError(f'Invalid feature map shape: {feature.shape}')

In [22]:
class EigenCAM(CAM):
    def get_heatmap(self, img):
        img_rgb = img.convert('RGB') if isinstance(img, Image.Image) else Image.fromarray(img).convert('RGB')
        tensor = self.prep(img_rgb)[None, ...].to(self.device)
        output = self.model(tensor)
        feature = self.feature['output']
        self._check(feature)

        _, _, vT = torch.linalg.svd(feature)
        v1 = vT[:, :, 0, :][..., None, :]
        cam = feature @ v1.repeat(1, 1, v1.shape[3], 1)
        cam = cam.sum(1)
        cam = cam - cam.min()
        cam = cam / (cam.max() - cam.min())

        cam = cam.detach().cpu().numpy().squeeze(0)
        cam = cv2.resize(cam, img_rgb.size)
        cam = np.uint8(255 * cam)
        heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)

        img_array = np.array(img_rgb)

        overlay = cv2.addWeighted(img_array, 0.7, heatmap, 0.3, 0)

        return output, overlay


In [27]:
def main(dataset_name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    model = mobilenet_v2(pretrained=True).eval()
    target_layer = 'features'

    if dataset_name.lower() == 'fashionmnist':
        dataset = FashionMNIST(root='./data', train=False, download=True, transform=preprocess)
    elif dataset_name.lower() == 'svhn':
        dataset = SVHN(root='./data', split='test', download=True, transform=preprocess)
    else:
        raise ValueError("Unsupported dataset. Choose 'FashionMNIST' or 'SVHN'.")

    cam_obj = EigenCAM(model, device, preprocess, layer_name=target_layer)

    if not os.path.exists('./drive/MyDrive/output1'):
        os.makedirs('./drive/MyDrive/output1')

    for i, (img, _) in enumerate(tqdm(dataset, desc="Processing images")):
        img = transforms.ToPILImage()(img).convert("RGB")
        output, overlay = cam_obj.get_heatmap(img)
        overlay_image = Image.fromarray(overlay)
        overlay_image.save(f'./drive/MyDrive/output1/heatmap_{i}.png')

        if i == 9:
            break

    print(f"Processed {i + 1} images.")

main('FashionMNIST')


Processing images:   0%|          | 0/10000 [00:00<?, ?it/s]


AttributeError: to

In [28]:
class EigenCAM(CAM):
    def get_heatmap(self, img_tensor):
        img_tensor = img_tensor.to(self.device)

        with torch.no_grad():
            output = self.model(img_tensor)
            feature = self.feature['output']
            self._check(feature)

            _, _, vT = torch.linalg.svd(feature)
            v1 = vT[:, :, 0, :][..., None, :]
            cam = feature @ v1.repeat(1, 1, v1.shape[3], 1)
            cam = cam.sum(1, keepdim=True)
            cam = cam - cam.min()
            cam = cam / (cam.max() - cam.min() + 1e-5)
            cam = cam.squeeze().detach().cpu().numpy()
            cam = cv2.resize(cam, (224, 224))
            cam = np.uint8(255 * cam)
            heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)

            return output, heatmap

In [29]:
import matplotlib.pyplot as plt
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [30]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    model = SimpleCNN().to(device)
    dataset = FashionMNIST(root='./data', train=False, download=True, transform=preprocess)
    model.eval()
    target_layer = 'features.3'
    cam_obj = EigenCAM(model, device, preprocess, layer_name=target_layer)

    output_dir = './drive/MyDrive/output2'
    os.makedirs(output_dir, exist_ok=True)

    for i in range(10):
        img, _ = dataset[i]
        img_unsqueezed = img.unsqueeze(0).to(device)
        _, heatmap = cam_obj.get_heatmap(img_unsqueezed)
        heatmap = cv2.resize(heatmap, (224, 224))
        img_pil = transforms.ToPILImage()(img)
        img_pil = img_pil.convert("RGB")
        plt.figure()
        plt.imshow(img_pil)
        plt.imshow(heatmap, cmap='jet', alpha=0.5)
        plt.colorbar()
        plt.title(f'Image Index: {i}')
        plt.axis('off')
        # plt.savefig(os.path.join(output_dir, f'heatmap_index_{i}.png'))
        plt.close()
        img_array = np.array(img_pil)
        overlay = cv2.addWeighted(img_array, 0.5, heatmap, 0.5, 0)
        overlay_image = Image.fromarray(overlay)
        overlay_image.save(os.path.join(output_dir, f'heatmap_{i}.png'))

    print(f"Generated heatmaps for 10 images in {output_dir}")

main()

Generated heatmaps for 10 images in ./drive/MyDrive/output2
